In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import ipyplot

from utils import *

In [ ]:
facedetect_data = Model.find(
    (Model.filename % "wedding*moose")
    | (Model.filename % "camera*roll")
    | (Model.filename % "grandpa")
    | (Model.filename % "glen*arbor")
).all()

len(facedetect_data)

In [ ]:
triton_client = grpcclient.InferenceServerClient(url=TRITON_SERVER_URL, verbose=False)

model_metadata = triton_client.get_model_metadata(
    model_name=DLIB_FACE_DESCRIPTOR_MODEL_NAME, model_version=MODEL_VERSION
)

model_config = triton_client.get_model_config(
    model_name=DLIB_FACE_DESCRIPTOR_MODEL_NAME, model_version=MODEL_VERSION
).config

input_names = [i.name for i in model_config.input]
output_names = [i.name for i in model_config.output]

In [ ]:
n = 0
max_images = 8
results = []

for m in facedetect_data:
    if m.faces[0].probability != 0:
        if n == max_images:
            break
        else:
            

In [ ]:
m.faces[0].probability

In [ ]:
n = 0
max_images = 8
results = []
for json_items in random.sample(facedetect_data, N_DEBUG):
    if json_items["faces"][0]["probability"] != 0:
        if n == max_images:
            break
        else:
            try:
                result = submit_to_dlib(
                    json_items,
                    input_names[0],
                    output_names,
                    request_id=json_items["filename"],
                )
                results.append(result)
                n += 1
            except Exception as e:
                print("There was an exception: {}".format(e))

descriptors = []
for result in results:
    for descriptor in result:
        descriptors.append(descriptor.as_numpy("face_descriptor"))

print("Example 128D descriptor vector:\n{}".format(descriptors[0]))

In [ ]:
## DEBUG
# facedetect_data = random.sample(facedetect_data, N_DEBUG)

In [ ]:
pbar = tqdm(
    total=len(facedetect_data),
    desc="Submitting photos to {} at {}".format(
        DLIB_FACE_DESCRIPTOR_MODEL_NAME, TRITON_SERVER_URL
    ),
)
futures = []
descriptors = []

with ThreadPoolExecutor() as executor:
    for chunk in chunked(facedetect_data, THREAD_CHUNKS):
        futures = []
        for json_items in chunk:
            pbar.update()
            if json_items["faces"][0]["probability"] != "0":
                try:
                    futures.append(
                        executor.submit(
                            submit_to_dlib,
                            json_items,
                            input_names[0],
                            output_names,
                            request_id=json_items["filename"],
                        )
                    )
                except Exception as e:
                    print(
                        "There was an exception: {}\n{}".format(
                            e, json_items["filename"]
                        )
                    )

        for future in as_completed(futures):
            try:
                for descriptor in results:
                    descriptors.append(descriptor.as_numpy("face_descriptor"))

            except Exception as e:
                for result in results:
                    for descriptor in result:
                        descriptors.append(descriptor.as_numpy("face_descriptor"))

In [ ]:
import dlib

dlib_descriptors = [dlib.vector(descriptor) for descriptor in descriptors]

labels = dlib.chinese_whispers_clustering(dlib_descriptors, 0.5)

idx = 0
for json_items in facedetect_data:
    if json_items["faces"][0]["probability"] != 0:
        for face in json_items["faces"]:
            face["person_id"] = labels[idx]
            idx += 1

In [ ]:
img = Image.open(
    "/data/photos/2004-2019-google-photos/175469_10150181882648327_506653326_8687323_1158060.jpeg"
)
img

In [ ]:
# facedetect_data

___